In [1]:
import graphlab

# Load a common image analysis dataset

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

This non-commercial license of GraphLab Create for academic use is assigned to phatthara.pis@gmail.com and will expire on November 01, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\phatt\AppData\Local\Temp\graphlab_server_1542898196.log.0


# Exploring the image data

In [ ]:
graphlab.canvas.set_target('ipynb')

In [6]:
image_train['image'].show

<bound method SArray.show of dtype: Image
Rows: 2005
['Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 3

# Train a classifier on the raw image pixels

In [9]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,target='label',features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1894

...

| 6         | 12       | 1.000000  | 2.996160     | 0.351109          | 0.297297            |

| 10        | 17       | 1.000000  | 3.985182     | 0.513728          | 0.540541            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Make a prediction with the simple model based on raw pixels

In [11]:
image_test[0:3]['image'].show

<bound method SArray.show of dtype: Image
Rows: 3
['Height: 32 Width: 32', 'Height: 32 Width: 32', 'Height: 32 Width: 32']>

In [12]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [13]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

# Evaluating raw pixel model on test data

In [14]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.481, 'auc': 0.7115035000000002, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     bird     |       dog       |  110  |
 |     dog      |       cat       |  284  |
 |     cat      |       cat       |  395  |
 |     bird     |    automobile   |  125  |
 |  automobile  |    automobile   |  625  |
 |     dog      |    automobile   |   98  |
 |     dog      |       dog       |  357  |
 |     cat      |       dog       |  213  |
 |  automobile  |       dog       |   88  |
 |  automobile  |       bird      |  111  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.47918262477116164, 'log_loss': 1.2260279167630885, 'precision': 0.

# Can we improve the model using deep features

In [15]:
len(image_train)

2005

In [ ]:
deep_learning_model = graphlab.load_model('imagenet_model')
image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [17]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features, let's train a classifier

In [18]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                         features=['deep_features'],
                                                         target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 6         | 13       | 0.250000  | 4.039919     | 0.800213          | 0.674797            |

| 10        | 17       | 0.250000  | 5.623646     | 0.885760          | 0.821138            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Apply the deep features model to first few images of test set

In [21]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [20]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

# Compute test_data accuracy of deep_features_model

In [22]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.78975, 'auc': 0.9416977500000008, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     dog      |    automobile   |   15  |
 |     bird     |       bird      |  804  |
 |  automobile  |       bird      |   22  |
 |     cat      |       bird      |   78  |
 |     bird     |       dog       |   61  |
 |  automobile  |       dog       |   7   |
 |     dog      |       bird      |   44  |
 |     cat      |    automobile   |   29  |
 |     cat      |       cat       |  669  |
 |     bird     |       cat       |  113  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7898906110026884, 'log_loss': 0.5518302909797681, 'precision': 0